In [1]:
#-------------------------------------------------------------------------------------------
#//////////////////////// ( KNJIŽNICE ) ////////////////////////////////////////////////////
#-------------------------------------------------------------------------------------------

import cv2 #                                    OpenCV - Knjižnica za strojni vid
import numpy as np #                            Numpy - zahtevne matematične operacije 
from IPython.display import clear_output #      Modul display.clear_output - brisanje rezultatov celic
import serial #                                 PySerial - serijska komunikacija
import time #                                   Time - knjižnica za štetje časa
import matplotlib.pyplot as plt #               Pyplot - knjižnica za grafe
from configparser import ConfigParser #         ConfigParser - branje .ini datotek
import csv #                                    Csv - pisanje .csv datotek




#-------------------------------------------------------------------------------------------
#/////////////////////// ( IZBERI VIDEO ) //////////////////////////////////////////////////
#-------------------------------------------------------------------------------------------

videoNo = "0012"
loc = "TestVideo/"
vidExtension = "__cv_camera01__video.avi"

path = loc + videoNo + vidExtension

path_override = "TestVideo/cebela1k.mp4"


#-------------------------------------------------------------------------------------------
#/////////////////////// ( INICIALIZACIJA ) ////////////////////////////////////////////////
#-------------------------------------------------------------------------------------------

config = ConfigParser()
config.read(loc + videoNo + '.ini')

display = config.getboolean('userSetings', 'display')

locdata = config.get('globalROI', 'loc')
gROIloc = [int(locdata[0:4]), int(locdata[5:9])]
sizedata = config.get('globalROI', 'size')
size = [int(sizedata[0:4]), int(sizedata[5:9])]

NoROIs = config.getint('localROI', 'norois')
ROIdata = np.zeros([NoROIs, 3])
for i in range(NoROIs):
    data = config.get('localROI', 'roi_' + str(i))
    ROIdata[i, 0] = data[0:4] #[X]
    ROIdata[i, 1] = data[5:9] #[Y]
    ROIdata[i, 2] = data[10:14] #[r]

print(ROIdata)


    
#-------------------------------------------------------------------------------------------
#/////////////////////// ( MATEMATIČNE NASTAVITVE ) ////////////////////////////////////////
#-------------------------------------------------------------------------------------------

xc, yc = 0, 0

#display = False

isShraniVideo = 0

beeR = 50

ROIstate = np.zeros(NoROIs)

pollinators = ['/', 'čebela', 'čmrlj', 'muha']

marks = []

#-------------------------------------------------------------------------------------------
#/////////////////////// ( FORMAT VIDEA ) ////////////////////////////////////////////
#-------------------------------------------------------------------------------------------

fourcc = cv2.VideoWriter_fourcc(*'XVID')
outvid = cv2.VideoWriter('output.avi',fourcc, 20.0, (1280, 720))
#outvid = cv2.VideoWriter('output.avi',fourcc, 20.0, (1280, 720), isColor = 0)

[[669. 372.  50.]
 [574. 542.  50.]
 [478. 245.  50.]
 [687.  98.  50.]
 [971. 346.  50.]
 [907. 558.  50.]]


In [2]:
#-------------------------------------------------------------------------------------------
#/////////////////////// ( DEFINICJE FUNKCIJ ) ////////////////////////////////////////////
#-------------------------------------------------------------------------------------------
#Funkcija za upragovljanje

def thresholdImage(iImage, iThreshold):
    """Upragovljanje: I>t"""
    oImage = 255 * np.array(iImage>iThreshold, dtype = "uint8")
    return oImage

# Funkcija za izris simbola križec na mesto x, y
def draw_crs1(image, x, y, color = (0,0,0), size=20):
    cv2.line(image, (x - size, y), (x + size, y), color, thickness=1, lineType=8, shift=0)
    cv2.line(image, (x, y - size), (x, y + size), color, thickness=1, lineType=8, shift=0)
    cv2.circle(image, (x, y), int(0.6*size), color, thickness=2, lineType=8, shift=0)
    cv2.putText(image, str(x), (x + 20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    cv2.putText(image, str(y), (x + 20, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)


# Funkcija za izris simbola krogci na mesto x, y
def draw_crs2(image, x, y, color = (0, 0, 0), size=20):
    cv2.circle(image, (x, y), int(size/4), color, -1)
    cv2.circle(image, (x, y), size, color, 2)
    cv2.putText(image, str(x - x0), (x + 20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    cv2.putText(image, str(y - y0), (x + 20, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    
# Funkcija za postavitev kurzorja na mesto x, y --> xc, yc
def set_crs(event, x, y, flags, params):
    global xc, yc
    if event == cv2.EVENT_LBUTTONDOWN:
        xc, yc = x, y
        #print("klik ", xc, yc)
        tocke_stare = np.array([[x, y]], dtype=np.float32)
        track = True


In [3]:
# Zajem videa
print("Starting video capture")
cap = cv2.VideoCapture(path_override)

totFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Odštevanje ozadja
fgbg = cv2.createBackgroundSubtractorMOG2(history = 500, varThreshold = 16, detectShadows = True)

tocke_stare = np.array([[]], dtype=np.float32)

print("Capture done")

#-------------------------------------------------------------------------------------------
#/////////////////////// ( GLAVNA ZANKA ) //////////////////////////////////////////////////
#-------------------------------------------------------------------------------------------

# Začetni pogoji
frameNo = 1
start_time = time.time()
prog_star = 0
track = False

print("---------------------------")
print("Processing:\n0%")


while(frameNo <= totFrames):
    
    #xc, yc = 0, 0
    
    if display:
        cv2.namedWindow('frame')
        cv2.setMouseCallback('frame', set_crs)
    
    # Definicija centra globalnega ROI območja in polovična vrednost horizontalne stranice
    C = int(locdata[0:4])
    D = int(locdata[5:9])
    L = int(int(sizedata[0:4])/2)
    H = int(int(sizedata[5:9])/2)
    
    
    #_______________ ( Razbitje posnetka, izluščenje barv ) ________________________
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    # Razbijemo video na sličice, obdelava 1 sličice vsako iteracijo
    ret, frame = cap.read()
    # Preoblikovanje posameznega frame-a (Za posnetke velikosti 1920x1080, drugače je potrebno vrstico zakomentirati)
    frame = cv2.resize(frame, (1280, 720))
    back2 = frame.copy()
    
    #Zanemarimo odšteto ozadje zunaj globalnega ROI območja
    back2[:,:(C-L)] = 0
    back2[:,(C+L):] = 0
    
    back2[:(D-H),:] = 0
    back2[(D+H):,:] = 0
    
    #back2[:,:300] = 0
    #back2[:,1200:] = 0
    sizeX, sizeY, cch = frame.shape
    back3 = fgbg.apply(back2)
    back4 = thresholdImage(back3.copy(), 200)
    
    
    
    #_______________ ( Filtri za zabris detajlov ) _________________________________
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    back = cv2.medianBlur(back4, 9)
    
    #_______________ ( Izračun in izris kontur ) _________________________________
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    kernel = np.ones((10, 10), np.uint8)
    slika = cv2.dilate(back,kernel,iterations = 1)
    im2, contours, hierarchy = cv2.findContours(slika,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    #cv2.drawContours(slika, contours, -1, (128,255,0), 3)
    for cnt in contours:
        if contours:
            cmax = max(contours, key = cv2.contourArea)
            cmin = min(contours, key = cv2.contourArea)
            if 15000 > cv2.contourArea(cnt) >= 50:
                #Določimo zgornje levo oglišče, ter obe stranici pravokotnika
                x,y,w,h = cv2.boundingRect(cmax)
                cv2.rectangle(slika,(x,y),(x+w,y+h),(255,0,0),2)
                xc = int((x+x+w)/2)
                yc = int((y+y+h)/2)
                #if display:
                    #print(xc)
                    #print(yc)
    

    
    
    
    #_______________ ( Iskanje položaja ) __________________________________________
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    for i in range(NoROIs):
        dist = np.sqrt((xc - ROIdata[i, 0])**2 + (yc - ROIdata[i, 1])**2)
        if dist - beeR <= ROIdata[i, 2]:
            ROIstate[i] = 1
        else:
            ROIstate[i] = 0
            
    
    if np.sum(ROIstate) > 0:
        mark = [str(frameNo)]
        #print("Opraševalec zaznan")
        for i in range(NoROIs):
            bitje = pollinators[int(ROIstate[i])]
            mark.append(bitje)
        marks.append(mark)
    
    
    #_______________ ( Prikaz matematičnih objektov ) ______________________________
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    t = round(time.time() - start_time)
    prog = int(100*frameNo/totFrames)
    
    if (not display) and (prog%10 == 0) and (prog != prog_star):
        print(str(prog) + "%")
        
    if display:
        txtX = 100
        txtY = 300
        cv2.putText(frame, "Time: {0:4.1f}s".format(t), (txtX, txtY), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        cv2.putText(frame, "FrameNo: " + str(frameNo), (txtX, txtY + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        cv2.putText(frame, "Progress: " + str(prog) + "%", (txtX, txtY + 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        
    
   
    
    #Izris globalnega ROI območja:
    cv2.rectangle(frame, (int(gROIloc[0] - size[0]/2), int(gROIloc[1] - size[1]/2)), (int(gROIloc[0] + size[0]/2), int(gROIloc[1] + size[1]/2)), (0, 0, 0), 2)
    
    
    #Izris lokalnih ROI območij:
    for i in range(NoROIs):
        if ROIstate[i] > 0:
            col = (0, 255, 0)
        else:
            col = (0, 0, 255)
        cv2.circle(frame, (int(ROIdata[i, 0]), int(ROIdata[i, 1])), int(ROIdata[i, 2]), col, 2)
        cv2.putText(frame, 'ROI' + str(i), (int(ROIdata[i, 0] + ROIdata[i, 2]), int(ROIdata[i, 1] - ROIdata[i, 2])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, col, 2)
    
    # Izris kurzorja:
    draw_crs1(frame, xc, yc)
    #_______________ ( Shranjevanje videa ) ______________________________________________
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    if isShraniVideo:
        outvid.write(frame)
    
    
    #_______________ ( Prikaz videa ) ______________________________________________
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    # Prikaz videa na zaslonu
    if display:
        cv2.imshow('frame',frame)
        cv2.imshow('background', back)

    
    # Inkrementacija/osveževanje spremenljivk
    frameNo = frameNo + 1
    prog_star = prog
    
    # Prekinitev glavne zanke ob pritisku "Esc"
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
        
        
        
#-------------------------------------------------------------------------------------------
#/////////////////////// ( KONEC GLAVNE ZANKE ) ////////////////////////////////////////////
#-------------------------------------------------------------------------------------------

print("Terminating algorithm")

cv2.destroyAllWindows() # Zapiranje vseh prikaznih oken
cap.release() # Prekinitev video zajema



#_______________ ( Beleženje rezultatov ) ______________________________________
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
print("--------------------------")
print("Flushing to output file")

with open(loc + videoNo + '.csv', mode='w') as outputFile:
    writer = csv.writer(outputFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    for i in range(len(marks)):
        writer.writerow(marks[i])

print("Flushing done")
print("---------------------------")


Starting video capture
Capture done
---------------------------
Processing:
0%
Terminating algorithm
--------------------------
Flushing to output file
Flushing done
---------------------------
